Assuming we already have SAT/AP scores and the high schools distances in the file, we would only look for high schools which are in California and we want to filter those schools out first. 

In [1]:
import pandas as pd
%matplotlib inline
import pylab as plt
import numpy as np
import scipy as sc
from scipy import stats

In [2]:
data = pd.read_csv('data/distances.csv')
data.head()

,campus,year,school,school_num,city,county,state,country,region,ethnicity,app_num,adm_num,enr_num,app_gpa,adm_gpa,enr_gpa,distance
0,Berkeley,1994.0,ABRAHAM LINCOLN HIGH SCHOOL,51520.0,Los Angeles,Los Angeles,California,USA,Los Angeles,All,14.0,NaN,NaN,3.620000,NaN,NaN,601648.0
1,Berkeley,1994.0,ABRAHAM LINCOLN HIGH SCHOOL,51520.0,Los Angeles,Los Angeles,California,USA,Los Angeles,Asian,8.0,NaN,NaN,3.620000,NaN,NaN,601648.0
2,Berkeley,1994.0,ABRAHAM LINCOLN HIGH SCHOOL,51520.0,Los Angeles,Los Angeles,California,USA,Los Angeles,Hispanic/ Latino,5.0,NaN,NaN,3.620000,NaN,NaN,601648.0
3,Berkeley,1994.0,ABRAHAM LINCOLN HIGH SCHOOL,52910.0,San Francisco,San Francisco,California,USA,San Francisco,All,58.0,8.0,7.0,3.682931,4.12125,4.088571,33037.0
4,Berkeley,1994.0,ABRAHAM LINCOLN HIGH SCHOOL,52910.0,San Francisco,San Francisco,California,USA,San Francisco,Asian,50.0,8.0,7.0,3.682931,4.12125,4.088571,33037.0


check the total number of high schools in our file 

In [3]:
tot_high = len(data['school'].unique())
tot_high

2843

In [4]:
data[data['state']=='California']
data.head()

,campus,year,school,school_num,city,county,state,country,region,ethnicity,app_num,adm_num,enr_num,app_gpa,adm_gpa,enr_gpa,distance
0,Berkeley,1994.0,ABRAHAM LINCOLN HIGH SCHOOL,51520.0,Los Angeles,Los Angeles,California,USA,Los Angeles,All,14.0,NaN,NaN,3.620000,NaN,NaN,601648.0
1,Berkeley,1994.0,ABRAHAM LINCOLN HIGH SCHOOL,51520.0,Los Angeles,Los Angeles,California,USA,Los Angeles,Asian,8.0,NaN,NaN,3.620000,NaN,NaN,601648.0
2,Berkeley,1994.0,ABRAHAM LINCOLN HIGH SCHOOL,51520.0,Los Angeles,Los Angeles,California,USA,Los Angeles,Hispanic/ Latino,5.0,NaN,NaN,3.620000,NaN,NaN,601648.0
3,Berkeley,1994.0,ABRAHAM LINCOLN HIGH SCHOOL,52910.0,San Francisco,San Francisco,California,USA,San Francisco,All,58.0,8.0,7.0,3.682931,4.12125,4.088571,33037.0
4,Berkeley,1994.0,ABRAHAM LINCOLN HIGH SCHOOL,52910.0,San Francisco,San Francisco,California,USA,San Francisco,Asian,50.0,8.0,7.0,3.682931,4.12125,4.088571,33037.0


check the total number of high schools in california 

In [5]:
num_cali_high = len(data[data['state']=='California']['school'].unique())
num_cali_high

860

In [6]:
data_cali_high = data[data['state']=='California']
data_cali_high['yield'] = data_cali_high['enr_num'] / data_cali_high['adm_num']
data_cali_high.describe()

/Users/Nelson/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,year,school_num,app_num,adm_num,enr_num,app_gpa,adm_gpa,enr_gpa,distance,yield
count,278029.000000,278029.000000,278029.000000,208973.000000,83776.000000,278029.000000,208973.000000,83776.000000,2.332610e+05,83776.000000
mean,2007.508990,52090.542490,24.390308,16.130845,12.035511,3.669844,3.884817,3.831920,3.345409e+05,0.482094
std,6.651732,17225.523927,31.352837,21.616506,15.655316,0.194324,0.227329,0.226885,2.617145e+05,0.200331
min,1994.000000,50003.000000,5.000000,3.000000,3.000000,2.468333,2.798462,2.782000,6.620000e+02,0.044776
25%,2002.000000,50735.000000,8.000000,5.000000,5.000000,3.545556,3.720667,3.690000,9.825300e+04,0.333333
50%,2008.000000,51395.000000,13.000000,9.000000,7.000000,3.683235,3.892222,3.842462,2.207980e+05,0.473684
75%,2013.000000,52774.000000,27.000000,18.000000,13.000000,3.807857,4.068571,3.994419,6.004540e+05,0.615385
max,2017.000000,998463.000000,544.000000,472.000000,295.000000,4.356667,4.495000,4.430000,1.348288e+06,1.000000


check how many NaN's are there

In [7]:
count_nan = len(data_cali_high) - data_cali_high.count()
count_nan

campus             0
year               0
school             0
school_num         0
city               0
county             0
state              0
country            0
region             0
ethnicity          0
app_num            0
adm_num        69056
enr_num       194253
app_gpa            0
adm_gpa        69056
enr_gpa       194253
distance       44768
yield         194253
dtype: int64

filter out the NaN data

In [8]:
data_cali_high = data_cali_high.dropna()
#data_cali_high = data_cali_high[pd.notnull(data_cali_high[['app_num', 'adm_num', 'app_gpa', 'adm_gpa', 'year']])]

X is to be fixed, it contains the features we want to include

In [9]:
X = data_cali_high[['app_num', 'adm_num', 'app_gpa', 'adm_gpa', 'year', 'distance']]
y = data_cali_high['yield']

In [10]:
def z_score(x):
    return (x - x.mean()) / x.std()

In [1]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

X_train = z_score(X_train)
y_train = z_score(y_train)
X_test  = z_score(X_test)
y_test  = z_score(y_test)

NameError: name 'X' is not defined

In [12]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [13]:
coeff_df = pd.DataFrame(regressor.coef_, X.columns, columns = ['coefficient'])

In [14]:
coeff_df

,coefficient
app_num,0.004270
adm_num,-0.431469
app_gpa,-0.077701
adm_gpa,0.298331
year,-0.086677
distance,-0.182316


In [15]:
y_pred = regressor.predict(X_test)

In [16]:
data_result = pd.DataFrame({'Actual Yield': y_test, 'Predicted Yield': y_pred}) 
data_result.head(10)

,Actual Yield,Predicted Yield
264025,-1.002975,-0.029656
266980,-0.823189,-0.637648
272496,0.155644,-0.124353
200621,0.938711,0.464500
215063,0.144888,-0.427768
86007,-0.532766,0.339291
180729,-1.549641,-2.168191
23456,0.714978,0.355855
18412,1.016157,0.105385
49512,-0.823189,0.112138


In [17]:
data_result.shape

(14579, 2)